In [1]:
#use "topfind";;
#thread;;
#require "nv_jupyter"
#require "jupyter"
#require "jupyter.notebook"
#require"jupyter.comm"
open Nv_jupyter.Driver;;
open Nv_jupyter.Show;;
open Nv_solution;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/opt/local/lib/ocaml/threads: added to search path
/opt/local/lib/ocaml/unix.cma: loaded
/opt/local/lib/ocaml/threads/threads.cma: loaded
/Users/gian/.opam/default/lib/num: added to search path
/Users/gian/.opam/default/lib/num/nums.cma: loaded
/Users/gian/.opam/default/lib/num-top: added to search path
/Users/gian/.opam/default/lib/num-top/num_top.cma: loaded
/opt/local/lib/ocaml/bigarray.cma: loaded
/opt/local/lib/ocaml/str.cma: loaded
/Users/gian/.opam/default/lib/bytes: added to search path
/Users/gian/.opam/default/lib/batteries: added to search path
/Users/gian/.opam/default/lib/batteries/batteries.cma: loaded
/Users/gian/.opam/default/lib/batteries/batteriesConfig.cmo: loaded
/Users/gian/.opam/default/lib/batteries/batteriesHelp.cmo: loaded
/Users/gian/.opam/default/lib/batteries/batteriesPrint.cmo: loaded
/Users/gian/.opam/default/lib/batteries/batteriesThread.cma: loaded
/Users/gian/.opam/default/lib/batteries/batteriesThread.cma: loaded
/Users/gian/.opam/default/lib/cudd: add

# NV: An intermediate language for network verification

NV is a functional language designed for modelling and verification of a network's routing behavior. Some of the design goals for NV are:
* Expressiveness. One should be able to express the semantics of common routing protocols such as BGP in a sufficiently detailed manner. At the same time, one should be able to express abstractions of protocols that are useful for verification.
* Tractable. NV's language constructs and semantics are chosen in a fashion that enables efficient verification, either through execution or using SMT solvers.
* Compact language, well-defined semantics. Having a small language, with conventional and hence well understood semantics simplifies rapid development and the maintenance of analyses.

## Modelling the control plane

To learn how to route traffic to different destinations (typically, IP prefixes) routers often rely on distributed routing protocols. According to these protocols, routers exchange *routing messages* (aka routes) containing information about a path to a given destination. Eventually, each router will select the *best* route among the ones it received; the route ranking algorithm depends on the protocol.

To define how a protocol works, one has to define the ingridients above:
1. The type (i.e. the components) of the routing messages exchanged
2. A *transfer function* defining how routes are transformed as they are propagated through the network.
3. A *merge function* defining how a router compares two routes to select the best one.

For instance, we can model BGP routes for a single destination as an optional record of 5 components. Lack of a value denotes the lack of a route. In the simple case, BGP will just increment the path length by 1. To implement more sophisticated routing policy a user may apply different changes to a route for each edge in the network.

The merge function simple selects the best route according to BGP's specification (i.e. higher local-pref, then shortest path, then lower med).

Below is an example of an NV program representing a hijack analysis over a network running BGP. In this network, node 4 (the purple node) announces a symbolic (i.e. an unknown) route and we want to see whether it can drive traffic away from the blue nodes which represent our network.
<img src="hijack.png" width="200"/>

```ocaml
type bgp = {length:int;
            lp:int;
            med:int;
            comms:set[int];
            origin: tnode}

type attribute = option[bgp]

let nodes = 5
let edges = {0=1;0=2;1=4;2=4;1=3;2=3;}

symbolic route : option[bgp]

let init (u : tnode) = match u with
| 0n -> Some {length=0; lp=100; med=80; comms={}; origin=0n}
| 4n -> route
| _ -> None

let trans (e: tedge) x =
  match x with
  | None -> None
  | Some b -> Some {b with length = b.length+1}

let isBetter x y =
  match (x,y) with
  | (None, _) -> false
  | (_, None) -> true
  | (Some b1, Some b2) ->
    if b1.lp > b2.lp then true
    else if b2.lp > b1.lp then false
    else if b1.length < b2.length then true
    else if b2.length < b1.length then false
    else if b1.med <= b2.med then true else false

let merge (u: tnode) x y =
  if isBetter x y then x else y

let assert (u : tnode) x =
  match x with
  | None -> false
  | Some b -> if !(u = 4n) then b.origin = 0n else true

```

We assert that for every node in our network the origin of the selected route should be node 0.
A simulation-based approach requires fixing the symbolic route to a single concrete route and hence verification is not under "all possible incoming routes". If no concrete value is specified for a symbolic value, NV will use a default value based on the type of the symbolic (in this case, the default value is ```None```).

In [2]:
let bgpExample = "/Users/gian/Documents/Princeton/networks/davidnv/nv/examples/bgp-paper.nv"
let bgpSol = compute [|"-inline";"-simulate"; |] bgpExample
let _ = checkAssertions bgpSol

val bgpExample : string =
  "/Users/gian/Documents/Princeton/networks/davidnv/nv/examples/bgp-paper.nv"


Processing /Users/gian/Documents/Princeton/networks/davidnv/nv/examples/bgp-paper.nv
Inlining took: 0.000546 secs to complete
Interpreted simulation took: 0.001544 secs to complete


val bgpSol : Nv_solution.Solution.t option =
  Some
   {Nv_solution.Solution.symbolics = <abstr>; labels = <abstr>;
    assertions = Some <abstr>; mask = None}


All assertions passed!

- : Jupyter_notebook.display_id = <abstr>


In [3]:
let _ = showSolutions bgpSol

Node 0 Some {comms = {}; length = 0u32; lp = 100u32; med = 80u32; origin = 0n; } Node 1 Some {comms = {}; length = 1u32; lp = 100u32; med = 80u32; origin = 0n; } Node 2 Some {comms = {}; length = 1u32; lp = 100u32; med = 80u32; origin = 0n; } Node 3 Some {comms = {}; length = 2u32; lp = 100u32; med = 80u32; origin = 0n; } Node 4 Some {comms = {}; length = 2u32; lp = 100u32; med = 80u32; origin = 0n; }

- : Jupyter_notebook.display_id = <abstr>


### Using the SMT for exhaustive verification

The NV simulator did not find a counterexample in the above case.
However, that does not mean that our network is "safe". In the presence of "unknown" factors, such as routes from external peers, verification is best done using an SMT-based approach. In this case, we will verify that the assertion holds against any possible route announced by node 4.

In [4]:
let bgpSol = compute [|"-inline";"-smt"; |] bgpExample
let _ = checkAssertions bgpSol

Processing /Users/gian/Documents/Princeton/networks/davidnv/nv/examples/bgp-paper.nv
Inlining took: 0.000756 secs to complete
Map unrolling took: 0.001392 secs to complete
Inlining took: 0.000137 secs to complete
Unbox options took: 0.000666 secs to complete
Flattening Tuples took: 0.001299 secs to complete
Partially Evaluating Network took: 0.001542 secs to complete
Encoding network took: 0.182708 secs to complete
Compiling query took: 0.006450 secs to complete
Z3 stats:
  :memory              3.39
 :time                0.02)
Slice 0 took: 0.240254 secs to complete


val bgpSol : Nv_solution.Solution.t option =
  Some
   {Nv_solution.Solution.symbolics = <abstr>; labels = <abstr>;
    assertions = Some <abstr>; mask = None}


Symbolic,Concrete Value
route,Some {comms = {}; length = 0u32; lp = 100u32; med = 80u32; origin = 8366u32; }
Assertion,Nodes
Passed,"0,4"
Failed,"1,2,3"


- : Jupyter_notebook.display_id = <abstr>


We can also display the solutions the SMT solver computed to better understand what's going on.

In [11]:
showSolutions ~asTable:true bgpSol

Node,Solution
0,Some {comms = {}; length = 0u32; lp = 100u32; med = 80u32; origin = 0u32; }
1,Some {comms = {}; length = 1u32; lp = 100u32; med = 80u32; origin = 8366u32; }
2,Some {comms = {}; length = 1u32; lp = 100u32; med = 80u32; origin = 8366u32; }
3,Some {comms = {}; length = 2u32; lp = 100u32; med = 80u32; origin = 8366u32; }
4,Some {comms = {}; length = 0u32; lp = 100u32; med = 80u32; origin = 8366u32; }


- : Jupyter_notebook.display_id = <abstr>


## Modelling the data plane

Beyond the control plane, it would be interesting to model other parts of the networking stack and compose the analyses to provide stronger end-to-end guarantees. Towards this goal, we are working on building a model of the dataplane as well.

We start with a basic model of the dataplane as an instance of the stable paths problem. In this instance of SPP:
- The type of "information" exchanged is sets of packets. 
- A packet is forwarded over an edge if: 1. according to the FIB of the node, this edge is the forwarding edge for the packet's destination, 2. there are no ACLs installed that prevent forwarding this packet. 
- Merging the information received from two different nodes simple amounts to taking the union of the sets.

The above SPP essentially computes for each node the packets that traversed it.

```ocaml
type prefix = (int,int)
type packet = {srcIp:prefix; dstIp:prefix}
type attribute = set[packet]

symbolic fib : ...

let trans e ps = 
  mapIte (fun p -> isForwarding e fib p.dstIp) (fun v -> true) (fun v -> false) ps
  
let merge u xs ys = xs union ys
```

  

```ocaml
include "utils.nv"

type prefix = (int,int)

type bgp = { nexthop: option[tedge];
             bgpad : int;
             lp : int;
             cost : int; }
type ribEntry = option[bgp]

type packet = { srcIp: int; dstIp: int; }
type packets = set[packet]

type attr = {dataplane:packets; controlplane:dict[prefix,ribEntry];}
type attribute = attr

let nodes = 4

let edges = {
  0=1;
  0=2;
  1=3;
  2=3;
 }

(* Data Plane functions *)
let mergePackets node x y = x union y

(* longestMatchingPrefix, should check whether p's headers match an
   entry in the rib and whether edge is the forwarding edge on that rib *)
let longestMatchingPrefix edge p rib =
  let dst = p.dstIp in
  match rib[(dst,32)] with
  | None -> false (*repeat 32 times...*)
  | Some route ->
    (match route.nexthop with
    | None -> false
    | Some fwd ->
     fwd = edge)

(* filter the set of packets *)
let transPackets edge rib ps =
  mapIf (fun p -> !(longestMatchingPrefix edge p rib)) (fun b -> false) ps

let initPackets node =
  match node with
  | 3n -> {{ srcIp = 1; dstIp=0; }}
  | _ -> {}

let transRoute edge b =
    match b with
    | None -> None
    | Some b ->
         Some ({nexthop= flipEdge edge; cost = b.cost + 1; bgpad=b.bgpad; lp=b.lp})

(* bgp attributes comparison *)
let betterBgp o1 o2 =
  if o1.lp > o2.lp then true
  else if o2.lp > o1.lp then false
  else if o1.cost < o2.cost then true
  else false

let pickOption f x y =
  match (x,y) with
  | (None, None) -> None
  | (None, Some _) -> y  | (Some _, None) -> x
  | (Some a, Some b) ->
    Some (if f a b then a else b)

let mergeRoute lab1 lab2 =
  pickOption (fun x y -> betterBgp x y) lab1 lab2

let initRoute node =
  let d = createDict None in
  match node with
  | 0n ->
     let b0 = {nexthop=None; bgpad=20; lp=100; cost=0;} in
     d[(0,32) := Some b0]
  | _ -> d

(* Combined functions *)

let merge node x y =
  let psx = x.dataplane in
  let psy = y.dataplane in
  let labx = x.controlplane in
  let laby = y.controlplane in
  {dataplane=mergePackets node psx psy;
   controlplane=combine mergeRoute labx laby}

let trans edge x =
  let ps = x.dataplane in
  let rib = x.controlplane in
  {dataplane = transPackets edge rib ps;
   controlplane = map (transRoute edge) rib}

let init node =
  {dataplane = initPackets node;
   controlplane = initRoute node}

let assert node x =
  match node with
  | 1n ->
     (
       let ps = x.dataplane in
       match ps[{ srcIp = 1; dstIp=0; }] with
       | true ->
          true
       | false -> false)
  | _ -> true
```

In [13]:
let dataplaneExample = "/Users/gian/Documents/Princeton/networks/davidnv/nv/examples/dataplane.nv"
let dataplaneSol = compute [|"-inline";"-simulate"|] dataplaneExample

val dataplaneExample : string =
  "/Users/gian/Documents/Princeton/networks/davidnv/nv/examples/dataplane.nv"


Processing /Users/gian/Documents/Princeton/networks/davidnv/nv/examples/dataplane.nv
Processing /Users/gian/Documents/Princeton/networks/davidnv/nv/examples/utils.nv
Inlining took: 0.002044 secs to complete
Interpreted simulation took: 0.019620 secs to complete


val dataplaneSol : Nv_solution.Solution.t option =
  Some
   {Nv_solution.Solution.symbolics = <abstr>; labels = <abstr>;
    assertions = Some <abstr>; mask = None}


In [14]:
checkAssertions dataplaneSol;
showSolutions ~asTable:true dataplaneSol

File "[14]", line 1, characters 0-28:
Warning 10: this expression should have type unit.


All assertions passed!

File "[14]", line 1, characters 0-28:
Warning 10: this expression should have type unit.


Node Solution 0 controlplane dataplane (0u32,32u32) Some {bgpad = 20u32; cost = 0u32; lp = 100u32; nexthop = None; } * None {(0u32,1u32)} 1 controlplane dataplane (0u32,32u32) Some {bgpad = 20u32; cost = 1u32; lp = 100u32; nexthop = Some 1~0; } * None {(0u32,1u32)} 2 controlplane dataplane (0u32,32u32) Some {bgpad = 20u32; cost = 1u32; lp = 100u32; nexthop = Some 2~0; } * None {} 3 controlplane dataplane (0u32,32u32) Some {bgpad = 20u32; cost = 2u32; lp = 100u32; nexthop = Some 3~1; } * None {(0u32,1u32)}

- : Jupyter_notebook.display_id = <abstr>


In [54]:
let file = ref ""

(* This function will be called with [json = `String text] when a message is received via a comm from Javascript code. *)
let receive_cell_text _ json = 
  let tmpFile, out = Filename.open_temp_file "nv" "file" in
  let str = Yojson.Safe.to_string json in
  let _, str = BatString.replace str "```ocaml" "" in
  let _, str = BatString.replace str "```" "" in
  Printf.fprintf out "%s" str;
  flush out;
  Format.printf "%s@." tmpFile;
  file := tmpFile;
  Format.printf "%s@." !file
  
(* Registers a comm for acception of messages from JavaScript code. *)
let target = Jupyter_comm.Manager.Target.create "comm-code" ~recv_msg:receive_cell_text
(* Gets a text of the first cell and sends it to the OCaml kernel. *)
let disp = Jupyter_notebook.display "text/html"

{|<script> 
comm = Jupyter.notebook.kernel.comm_manager.new_comm('comm-code');
var txt = Jupyter.notebook.get_cell(2).get_text();
comm.send(txt);
</script>|} 

val file : string ref = {contents = ""}


val receive_cell_text : 'a -> Yojson.Safe.t -> unit = <fun>


val target : Jupyter_comm.Manager.Target.t = <abstr>


val disp : string -> Jupyter_notebook.display_id = <fun>


- : string =
"<script> \ncomm = Jupyter.notebook.kernel.comm_manager.new_comm('comm-code');\nvar txt = Jupyter.notebook.get_cell(2).get_text();\ncomm.send(txt);\n</script>"
